In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install tortoise-tts scipy

Looking in indexes: https://download.pytorch.org/whl/cu118


In [2]:
!pip install deepspeed

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.9 MB/s eta 0:00:00
  Created wheel for deepspeed: filename=deepspeed-0.15.4-py3-none-any.whl size=1527834 sha256=f839be2a146a608f12e14d784fc23bf760c1f2719aeadd0f88eeb661d2ab1da6
  Stored in directory: /root/.cache/pip/wheels/74/bc/b6/836d7c3e3093e25502fa9248e0be9e943db245f2806ba1cd19
Successfully built deepspeed


In [3]:
import os
import numpy as np
import torch
from tortoise.api import TextToSpeech
from scipy.io.wavfile import write
import json
from google.colab import drive  # Google 드라이브 마운트

# Google 드라이브 마운트
drive.mount('/content/drive')

# Google 드라이브에 저장할 루트 폴더 설정
output_root_folder = '/content/drive/MyDrive/Colab_Notebooks/tortoise_tts_high_quality_novel'
os.makedirs(output_root_folder, exist_ok=True)

# A100 GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    print(f"GPU 활성화: {gpu_name}")
    if "A100" not in gpu_name:
        print("⚠️ GPU가 A100이 아닙니다. 런타임 유형 설정에서 'A100 GPU'로 변경하세요.")
else:
    print("GPU를 사용할 수 없습니다. CPU로 실행됩니다.")

# 저장된 임베딩 로드 및 Tensor로 변환
embedding_path = "conditioning_latents_Mark.npz"  # 본인 경로에 맞게 설정
loaded_latents = np.load(embedding_path)
conditioning_latents = [torch.tensor(loaded_latents[key]).to(device) for key in loaded_latents]

# Tortoise TTS 모델 초기화 (A100 GPU 최적화 설정)
tts = TextToSpeech(use_deepspeed=True, kv_cache=True, half=True, device=device)

# JSON 파일 경로 설정
json_path = "novel.json"  # 본인 경로에 맞게 설정

# JSON 파일 읽기
with open(json_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# JSON 데이터를 바탕으로 음성 파일 생성 및 저장
for category in data["categories"]:
    category_name = category["category_name"]
    for level in category["levels"]:
        level_number = level["level"]

        # level이 6에서 10 사이인 경우에만 음성 변환 수행
        if level_number < 6 or level_number > 10:
            continue

        for script in level["scripts"]:
            title = script["title"]
            contents = script["contents"]

            # Google 드라이브에 카테고리/레벨/타이틀 경로 생성
            output_folder = os.path.join(output_root_folder, category_name, f"level_{level_number}", title)
            os.makedirs(output_folder, exist_ok=True)

            # 각 문장을 한 줄씩 읽어 음성 변환 및 저장
            for i, text in enumerate(contents, 1):
                if not text.strip():
                    continue

                # 출력 파일 경로
                output_path = os.path.join(output_folder, f"{title}_line_{i}.wav")

                # 파일이 이미 존재하면 건너뛰기
                if os.path.exists(output_path):
                    print(f"이미 생성된 파일: {output_path}. 건너뜁니다.")
                    continue

                # 텍스트를 화자 샘플의 스타일로 음성 변환 (High Quality 설정)
                pcm_audio = tts.tts_with_preset(
                    text, conditioning_latents=conditioning_latents, preset="high_quality"
                )

                # PCM 데이터를 numpy 배열로 변환 후 최종 WAV 파일로 저장
                write(output_path, 22050, np.array(pcm_audio, dtype=np.float32))
                print(f"'{text}' 문장이 '{output_path}'로 저장되었습니다.")

print("레벨 6~10에 대한 텍스트 파일의 음성 변환이 완료되었습니다.")


Mounted at /content/drive
GPU 활성화: NVIDIA A100-SXM4-40GB


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/2.11k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:460: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_location="cp

...

clvp2.pth:   0%|          | 0.00/976M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/tortoise/api.py:229: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.clvp.load_state_dict(torch.load(get_model_path('clvp2.pth', m

vocoder.pth:   0%|          | 0.00/391M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/tortoise/api.py:233: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.vocoder.load_state_dict(torch.load(get_model_path('vocoder.pt

이미 생성된 파일: /content/drive/MyDrive/Colab_Notebooks/tortoise_tts_high_quality_novel/novel/level_6/The Firekeeper’s Quest/The Firekeeper’s Quest_line_1.wav. 건너뜁니다.
이미 생성된 파일: /content/drive/MyDrive/Colab_Notebooks/tortoise_tts_high_quality_novel/novel/level_6/The Firekeeper’s Quest/The Firekeeper’s Quest_line_2.wav. 건너뜁니다.
이미 생성된 파일: /content/drive/MyDrive/Colab_Notebooks/tortoise_tts_high_quality_novel/novel/level_6/The Firekeeper’s Quest/The Firekeeper’s Quest_line_3.wav. 건너뜁니다.
이미 생성된 파일: /content/drive/MyDrive/Colab_Notebooks/tortoise_tts_high_quality_novel/novel/level_6/The Firekeeper’s Quest/The Firekeeper’s Quest_line_4.wav. 건너뜁니다.
이미 생성된 파일: /content/drive/MyDrive/Colab_Notebooks/tortoise_tts_high_quality_novel/novel/level_6/The Firekeeper’s Quest/The Firekeeper’s Quest_line_5.wav. 건너뜁니다.
이미 생성된 파일: /content/drive/MyDrive/Colab_Notebooks/tortoise_tts_high_quality_novel/novel/level_6/The Firekeeper’s Quest/The Firekeeper’s Quest_line_6.wav. 건너뜁니다.
이미 생성된 파일: /content/drive/MyDrive/